In [4]:
import couchdb
import dotenv
import os

In [5]:
dotenv.load_dotenv()

True

In [6]:
COUCHDB_HOST = os.environ['COUCHDB_HOST']
COUCHDB_PORT = os.environ['COUCHDB_PORT']
COUCHDB_USERNAME = os.environ['COUCHDB_USERNAME']
COUCHDB_PASSWORD = os.environ['COUCHDB_PASSWORD']

In [7]:
instance_url = f"http://{COUCHDB_USERNAME}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}:{COUCHDB_PORT}"
server = couchdb.Server(instance_url)

In [23]:
class CouchDB:
    def __init__(self, host=COUCHDB_HOST, port=COUCHDB_PORT, 
                 username=COUCHDB_USERNAME, password=COUCHDB_PASSWORD):
        self.host = host,
        self.port = port,
        self.username = username
        self.password = password

        self.instance_url = f"http://{self.username}:{self.password}@{self.host}:{self.port}"
        self.server = "Server Disconnected"
    
    def connect(self):
        self.server = couchdb.Server(self.instance_url)
        print(self.server)
        
    def __repr__(self):
        return f"{self.server}"
        
    def connect(self):
        self.server = couchdb.Server(self.instance_url)
        print(self.server)
        
    def upload_document(self, db, data, verbose=False):
        doc_id, doc_rev = db.save(data)
        if verbose:
            print(f"Document uploaded with ID: {doc_id}", end='\r')
    
    def create_database(self, dbname):
        try:
            db = self.server.create(dbname)
            print(f"Database '{dbname}' created successfully.")
            return db
        except couchdb.http.PreconditionFailed:
            print(f"Database '{dbname}' already exists.")
            return self.server[dbname]

    def get_document(self, db, doc_id):
        try:
            doc = db[doc_id]
            return doc
        except couchdb.http.ResourceNotFound:
            print(f"Document with ID '{doc_id}' not found.")
            return None
    
    def delete_document(self, db, doc_id):
        try:
            doc = db[doc_id]
            db.delete(doc)
            print(f"Document with ID '{doc_id}' deleted successfully.")
        except couchdb.http.ResourceNotFound:
            print(f"Document with ID '{doc_id}' not found.")
    
    def update_document(self, db, doc_id, updated_data):
        doc = self.get_document(db, doc_id)
        if doc:
            doc.update(updated_data)
            db.save(doc)
            print(f"Document with ID '{doc_id}' updated successfully.")
        else:
            print(f"Document with ID '{doc_id}' not found.")

    def list_databases(self):
        return self.server.all_dbs()

    def list_documents(self, db):
        return [doc for doc in db.view("_all_docs")]



In [24]:
CouchDB().connect()

<Server "http://('172.26.134.180',):('5984',)">


In [17]:
db_name = 'twitter'
# create a twitter database if not exist
if db_name in server:
    db = server[db_name]
    print(f"Database '{db_name}' already exists")
else:
    db = server.create(db_name)
    print(f"Database '{db_name}' created")

Database 'twitter' already exists


In [19]:
# check db object
type(db)

couchdb.client.Database

In [95]:
def upload_document(data: dict, db: couchdb.client.Database=db):
    """
    Args:
        data (dict): dictionary like object
        db (couchdb.client.Database): target CouchDB database
    Return:
        _id: documentation id
        rev: revision id
    """
    try:
        doc_id, doc_rev = db.save(data)
        print(f"Document uploaded with ID: {doc_id}", end='\r')
    except couchdb.http.ResourceConflict:
        # If the document already exists, update it with the latest data
        doc_id = data['_id']
        doc_rev = db[doc_id]['_rev']
        data['_rev'] = doc_rev
        db.save(data)
        print(f"Document updated with ID: {doc_id}", end='\r')

In [23]:
import pandas as pd
import numpy as np
from pathlib import Path

In [71]:
data_path = Path()/".."/"data"
data_path

PosixPath('../data')

In [72]:
df = pd.read_json(data_path/'twitter_cleaned_00.json')
df.tail()

,_id,author_id,created_at,location,possible_suburb,text
1907371,1494150729711644672,1359453724729507840,2022-02-17 03:02:42+00:00,,,@rebelsbynight @taeuchi @xiuminnimida @phekms
1907372,1494150729711644672,1359453724729507840,2022-02-17 03:02:42+00:00,,,@rebelsbynight @taeuchi @xiuminnimida @phekms
1907373,1494150729711644672,1359453724729507840,2022-02-17 03:02:42+00:00,,,@rebelsbynight @taeuchi @xiuminnimida @phekms
1907374,1494150729711644672,1359453724729507840,2022-02-17 03:02:42+00:00,,,@rebelsbynight @taeuchi @xiuminnimida @phekms
1907375,1494275408849928192,1450989035644592128,2022-02-17 11:35:32+00:00,,,@rebelsbynight @emmaolivia9714 @PigCaSaurus @a...


In [73]:
df.shape

(1907376, 6)

In [78]:
df.drop_duplicates('_id')

,_id,author_id,created_at,location,possible_suburb,text
0,1491571754170421248,1234462200623816704,2022-02-10 21:43:49+00:00,,,@von_battenberg Sorry for the hassle. Kindly s...
140,1491734528779763712,62407961,2022-02-10 16:51:31+00:00,,,@HerbsandDirt @iabfe medical FREEDOM for me
158,1491692008666439680,999027989898711040,2022-02-10 14:03:50+00:00,,,Vote republican in November please! https://t....
196,1491898152068448256,1071368432224940032,2022-02-10 04:43:55+00:00,,,@8bbbsnfhjd I'm pretty sure he's got other vid...
208,1491721359587627008,352311036,2022-02-10 22:44:37+00:00,,,@766Aka @meeez60356378 DM FOR SHOOTOUT\nFollow...
...,...,...,...,...,...,...
1907299,1494127131877863424,3060756174,2022-02-17 01:42:20+00:00,,,@MemoryTuxedo1 Comings back later mama!
1907306,1494149176258547712,1229633724272132096,2022-02-17 01:45:14+00:00,,,@Music_Kitts Very sweet!
1907316,1494125057982877696,96924694,2022-02-17 02:05:07+00:00,,,"Just posted a photo @ The Spit, Main Beach htt..."
1907366,1494150729711644672,1359453724729507840,2022-02-17 03:02:42+00:00,,,@rebelsbynight @taeuchi @xiuminnimida @phekms


In [75]:
df.loc[0].text

'@von_battenberg Sorry for the hassle. Kindly send us an email at sm.groupon.au@groupon.com, and we will take this up further.\\n\\n- Roy'

In [79]:
df._id = df._id.astype(str)
df.author_id = df.author_id.astype(str)
df.created_at = df.created_at.apply(lambda x: x.isoformat())

In [88]:
from tqdm.notebook import tqdm

In [ ]:
for _, row in tqdm(df.drop_duplicates('_id').iterrows(), total=len(df.drop_duplicates('_id'))):
    data = row.to_dict()
    upload_document(data)

In [68]:
df.shape

(1907376, 6)

In [59]:
upload_document(data)

Document uploaded with ID: {_id}
